In [1]:
import pandas as pd
import requests as rq
import json
from datetime import datetime as dt
import ast

In [2]:
api = rq.get("https://api.cartolafc.globo.com/atletas/mercado")

In [3]:
market = json.loads(api.text)

In [4]:
market.keys()

dict_keys(['atletas', 'clubes', 'posicoes', 'status'])

In [5]:
round_num = '09'

In [6]:
roundN = pd.DataFrame(market["atletas"])

In [7]:
with open("../../status_id.json") as file:
    status_id = json.load(file)
with open("../../positions.json") as file:
    pos_id = json.load(file)

In [8]:
roundN = roundN[["rodada_id", "atleta_id", "nome", "apelido", "posicao_id", "clube_id", "scout", "pontos_num", "preco_num", "variacao_num"]]
roundN.columns = ["round_num", "player_id", "player_name", "player_nickname", "position", "team_id", "scout", "total_score", "current_price", "price_delta"]

In [9]:
roundN.position = roundN.position.apply(lambda x: pos_id[str(x)])

In [10]:
def scout_parser(scout, col_names):
    scout_dict = ast.literal_eval(scout)
    scout_list = []
    for col in col_names:
        try:
            el = scout_dict[col]
        except:
            el = 0
        scout_list.append(el)
    return pd.DataFrame(columns = col_names, data = [scout_list]).iloc[0]

In [11]:
col_names = ["G", "A", "FT", "FD", "FF", "FS", "PP", "I", "PE", "SG", "DP", "DD", "RB", "GC", "CV", "CA", "GS", "FC"]
roundN[col_names] = roundN.scout.apply(lambda x: scout_parser(str(x), col_names))
roundN = roundN[["round_num", "player_id", "player_name", "player_nickname", "position", "team_id", "G", "A", "FT", "FD", "FF", "FS", "PP", "I", "PE", "SG", "DP", "DD", "RB", "GC", "CV", "CA", "GS", "FC", "total_score", "current_price", "price_delta"]]

In [12]:
roundN.head(10)

,round_num,player_id,player_name,player_nickname,position,team_id,G,A,FT,FD,...,DD,RB,GC,CV,CA,GS,FC,total_score,current_price,price_delta
0,9,36886,Eugênio Machado Souto,Geninho,TEC,314,0,0,0,0,...,0,0,0,0,0,0,0,0.70,5.75,-0.66
1,9,37657,Eduardo Luís Abonizio de Souza,Edu Dracena,ZAG,275,0,0,0,0,...,0,1,0,0,0,0,0,6.30,7.22,0.51
2,9,38001,Anderson Pedro Nunes da Silva Campos,Anderson,GOL,265,0,0,0,0,...,1,0,0,0,0,2,0,0.00,2.08,0.00
3,9,37246,Ney Franco da Silveira Júnior,Ney Franco,TEC,315,0,0,0,0,...,0,0,0,0,0,0,0,3.89,8.80,0.56
4,9,37281,Luis Antônio Venker de Menezes,Mano Menezes,TEC,283,0,0,0,0,...,0,0,0,0,0,0,0,0.81,9.07,-0.77
5,9,38311,Márcio Rodrigues Araújo,Márcio Araújo,MEI,315,0,0,0,2,...,0,21,0,0,0,0,6,1.70,8.85,0.30
6,9,37333,Marcelo Ribeiro Cabo,Marcelo Cabo,TEC,341,0,0,0,0,...,0,0,0,0,0,0,0,2.11,5.21,-0.12
7,9,37604,Paulo André Cren Benini,Paulo André,ZAG,293,0,0,0,0,...,0,3,0,0,1,0,6,1.50,3.89,-0.02
8,9,37646,Ebert William Amâncio,Betão,ZAG,314,0,0,0,0,...,0,11,0,0,4,0,7,0.00,3.14,-0.38
9,9,37662,Jonathan Cícero Moreira,Jonathan,LAT,293,0,0,0,1,...,0,3,0,0,1,0,3,-0.30,3.68,-1.03


In [13]:
roundN.to_csv("2019_round"+ round_num + "_scouts.csv", index=False)

In [18]:
status = roundN[["rodada_id", "atleta_id", "status_id"]]

In [19]:
status.rodada_id = status.rodada_id + 1
now = dt.now()
status["query_time"] = dt(now.year, now.month, now.day, now.hour, now.minute, now.second)
status.status_id = status.status_id.apply(lambda x: status_id[str(x)])

/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
status = status[["rodada_id", "query_time", "atleta_id", "status_id"]]
status.columns = ["round_num", "query_time", "player_id", "status_id"]
status.head()

,round_num,query_time,player_id,status_id
0,9,2019-06-12 17:57:12,98832,Provável
1,9,2019-06-12 17:57:12,100840,Nulo
2,9,2019-06-12 17:57:12,100742,Nulo
3,9,2019-06-12 17:57:12,100741,Nulo
4,9,2019-06-12 17:57:12,94968,Provável


In [21]:
filename = "../status/2019_round"
if status.round_num[0] < 10:
    filename = filename + "0" + str(status.round_num[0]) + "_status.csv"
else:
    filename = filename + str(status.round_num[0]) + "_status.csv"

status.to_csv(filename, index=False)